# Assessing Data Quality

Import librerie

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Lettura del dataset

In [2]:
df = pd.read_csv("dataset/customer_supermarket.csv", sep="\t",index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471910 entries, 0 to 541909
Data columns (total 8 columns):
BasketID           471910 non-null object
BasketDate         471910 non-null object
Sale               471910 non-null object
CustomerID         406830 non-null float64
CustomerCountry    471910 non-null object
ProdID             471910 non-null object
ProdDescr          471157 non-null object
Qta                471910 non-null int64
dtypes: float64(1), int64(1), object(6)
memory usage: 32.4+ MB


In [3]:
df.head()

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
0,536365,01/12/10 08:26,"2,55",17850.0,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,01/12/10 08:26,"3,39",17850.0,United Kingdom,71053,WHITE METAL LANTERN,6
2,536365,01/12/10 08:26,"2,75",17850.0,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8
3,536365,01/12/10 08:26,"3,39",17850.0,United Kingdom,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,01/12/10 08:26,"3,39",17850.0,United Kingdom,84029E,RED WOOLLY HOTTIE WHITE HEART.,6


## Missing values

Verififichiamo la presenza di missing values all'interno del dataset. In questa fase rileviamo i problemi, in seguito modificheremo e sostituiremo gli attributi mancanti

In [4]:
df.isnull().any()

BasketID           False
BasketDate         False
Sale               False
CustomerID          True
CustomerCountry    False
ProdID             False
ProdDescr           True
Qta                False
dtype: bool

Possiamo osservare che ci sono missing values per gli attributi CustomerID e ProdDesc. Verifichiamo quanti sample sono affetti da questa mancanza

### CostumerID

In [5]:
df[df['CustomerID'].isnull()]

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
1444,536544,01/12/10 14:32,"2,51",NaN,United Kingdom,21774,DECORATIVE CATS BATHROOM BOTTLE,2
1446,536544,01/12/10 14:32,"1,66",NaN,United Kingdom,21787,RAIN PONCHO RETROSPOT,2
1447,536544,01/12/10 14:32,"1,66",NaN,United Kingdom,21790,VINTAGE SNAP CARDS,9
1448,536544,01/12/10 14:32,"2,51",NaN,United Kingdom,21791,VINTAGE HEADS AND TAILS CARD GAME,2
1450,536544,01/12/10 14:32,"0,43",NaN,United Kingdom,21802,CHRISTMAS TREE HEART DECORATION,9
...,...,...,...,...,...,...,...,...
541532,581498,09/12/11 10:26,"4,13",NaN,United Kingdom,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,1
541534,581498,09/12/11 10:26,"3,29",NaN,United Kingdom,85049a,TRADITIONAL CHRISTMAS RIBBONS,5
541538,581498,09/12/11 10:26,"4,96",NaN,United Kingdom,85150,LADIES & GENTLEMEN METAL SIGN,1
541539,581498,09/12/11 10:26,"10,79",NaN,United Kingdom,85174,S/4 CACTI CANDLES,1


In [6]:
len(df[df['CustomerID'].isnull()])

65080

Il numero totale di missin customerId è 65080, però questa mancanza è replicata varie volte. Per diversi sample con lo stesso basketid manca anche il customerID, quindi ci si riferisce sempre alla stessa persona. Vediamo quante sono i BaskedID singoli

In [7]:
len(df[df['CustomerID'].isnull()].BasketID.unique())

2437

In [8]:
len(df.BasketID.unique())

24627

I singoli basketId sono 2437 sul totale di 24627

La mancanza del costumerID è importante, perchè non ci permette di identificare lo stesso cliente. Per recuperalo potremo controllare se nei sample con gli stessi BasketID di quelli mancanti ci sono i dati, creare degli ID nuovi se non si riescono a recuperare oppure eliminare i sample


Vediamo se in alcuni casi il CustomerID è recuperabile

In [9]:

count=0
for elem in df[df['CustomerID'].isnull()].BasketID.unique():
    #Prendiamo dal dataset tutti gli elementi con lo stesso BasketID
    df_loc_equal_basketID=df[df['BasketID']==elem]
    #Controlliamo se qualcuno di questi elementi abbia il CustomerID
    if len(df_loc_equal_basketID[df_loc_equal_basketID['CustomerID'].notnull()])>0:
        print("BasketId:"+elem+"_____Numero di elementi:"+str(len(df_loc_equal_basketID)))
        print("Numero di elementi non null: "+len(df_loc_equal_basketID[df_loc_equal_basketID['CustomerID'].notnull()]))
        count+=1
print("Totale elementi: "+str(count))

Totale elementi: 0


Di conseguenza non è possibile recuperare nessun CostumerId pertanto si dovranno creare degli ID aggregando anche i BasketID oppure eliminare i sample

### ProdDesc

In [10]:
df[df['ProdDescr'].isnull()]

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
1986,536546,01/12/10 14:33,0,NaN,United Kingdom,22145,NaN,1
2024,536550,01/12/10 14:34,0,NaN,United Kingdom,85044,NaN,1
2025,536553,01/12/10 14:35,0,NaN,United Kingdom,37461,NaN,3
4347,536764,02/12/10 14:42,0,NaN,United Kingdom,84952C,NaN,-38
4348,536765,02/12/10 14:43,0,NaN,United Kingdom,84952C,NaN,19
...,...,...,...,...,...,...,...,...
522162,580381,02/12/11 17:58,0,NaN,United Kingdom,21758,NaN,-9
535322,581199,07/12/11 18:26,0,NaN,United Kingdom,84581,NaN,-2
535326,581203,07/12/11 18:31,0,NaN,United Kingdom,23406,NaN,15
535332,581209,07/12/11 18:35,0,NaN,United Kingdom,21620,NaN,6


In [11]:
len(df[df['ProdDescr'].isnull()])

753

La mancanza di valori per la descrizione del prodotto non ci interessa più di tanto, l'identificativo del prodotto è più rilevante, sono due attributi doppi. Per recuperalo potremo controllare se nei sample con gli stessi Productid di quelli mancanti ci sono i dati

Cerchiamo di recuperare la descrizione dei prodotti.

In [14]:
retrievable_count=0
error_count=0
for elem in df[df['ProdDescr'].isnull()].ProdID.unique():
    df_loc_equal_ProdID=df[df['ProdID']==elem]
    
    if len(df_loc_equal_ProdID[df_loc_equal_ProdID['ProdDescr'].notnull()])>0:
        # Recupero descrizione a partire dal primo prodotto della lista
        retrived_ProdDescr=df_loc_equal_ProdID[df_loc_equal_ProdID['ProdDescr'].notnull()].iloc[0].ProdDescr

        #Vediamo se le descrizioni sono tutte uguali saranno nan oppure una descrizione del prodotto
        if(len(df_loc_equal_ProdID['ProdDescr'].unique())>2):
            error_count+=1

        #DEBUG
        #print("ProdId: "+elem+" ProdDescr: "+retrived_ProdDescr)
        
        retrievable_count+=1
print("Totale elementi sostituibili: "+str(retrievable_count))
print("Errori trovati: "+str(error_count))

Totale elementi sostituibili: 541
Errori trovati: 76


Del dataframe creato si possono sostituire 541 elementi su 753 ma di questi 76 non sono coerenti tra di loro (stesso ProdID ma diversa ProdDescr). In ogni caso la descrizione nel prodotto è inutile quindi possiamo evitare di processarla e droppare la colonna nella data transformation

## Outliers

In questa sezione ci occupiamo del rilevamento degli outliers

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471910 entries, 0 to 541909
Data columns (total 8 columns):
BasketID           471910 non-null object
BasketDate         471910 non-null object
Sale               471910 non-null object
CustomerID         406830 non-null float64
CustomerCountry    471910 non-null object
ProdID             471910 non-null object
ProdDescr          471157 non-null object
Qta                471910 non-null int64
dtypes: float64(1), int64(1), object(6)
memory usage: 52.4+ MB


### BasketID ✔️ 
Il basketID è un nominale vediamo da che caratteri ed è composto da lettere maiuscole e numeri (Vedi DU-data_distribution). In questo caso *NON* abbiamo sample da eliminare 

### BasketDate ✔️ 
Indica una data non abbiamo valori particolari da eliminare (Vedi DU-data_distribution)

### ProID ✔️ 
è nominale, verifichiamo da che caratteri è composto solo da numeri e lettere maiuscole. Non ci sono problemi con questo attributo

### ProdDescr ✔️ 
Abbiamo giuà visto che alcuni valori sono null, si può verificare se ci sono descrizioni diverse con lo stesso productID ma è un attributo poco significativo come abbiamo detto quindi non è significativo per ora tenerlo in considerazione.

### Sale ⚠️
è un object ma dovrebbe essere float, In questo caso abbiamo dei prezzi negativi che sono stati inseriti per dei debiti. Da verificare se devono venire inseriti, eliminati o se ci sono altri dati ambigui. _Da verificare_

### CustomerID ⚠️
Abbiamo già visto che alcuni valori sono null, in questo caso possiamo o rimuoverli o creare degli id. Non possono essere recuperati questi valori perchp negli stessi basket id non ci sono i nomi.

### CustomerCountry ⚠️
è categorico, ci sono dei valori unspecified, ma non è significativo toglierli. _Da verificare_

### Quantità ⚠️
Nell'intervallo si vede che ci sono dei valori negativi e vanno sistemati._Da verificare_
